In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from rich import print
import warnings
import datetime
from astropy.time import Time
import os

In [2]:
def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

In [3]:
def angularSeparation(ra1, dec1, ra2, dec2):

    d2r = np.pi/180.
    ra2deg = 1./d2r

    d1 = dec1*d2r
    d2 = dec2*d2r

    r1 = ra1*d2r
    r2 = ra2*d2r

    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))

    return r*ra2deg

In [4]:
header_data = pd.read_csv('../ref/mislike_image_header_table.csv')
catalog_filename = '../ref/catalog_nd_daostarfinder.parquet'
catalog_file = pq.ParquetFile(catalog_filename)

In [5]:
def make_target_table(objid_list,morphology=''):
    table = pd.DataFrame()
    for objid in objid_list:
        eclipse = int(str(objid)[-5:])
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',objid)]).to_pandas()
        this_star['morphology'] = morphology
        table = pd.concat([table,this_star[["eclipse","obj_id","ra","dec","morphology"]]])
    return table

In [7]:
gfcat_corpus = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_gifs/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_corpus = pd.concat([gfcat_corpus,table])
print(f'{len(gfcat_corpus)} sources found in general search')

gfcat_targeted = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_known_jpg/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_targeted = pd.concat([gfcat_targeted,table])
print(f'{len(gfcat_targeted)} sources found in targeted search')

gfcat_fuv_matched = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_fuv_matched/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_fuv_matched = pd.concat([gfcat_fuv_matched,table])
print(f'{len(gfcat_fuv_matched)} sources found in matched FUV targets')



865 sources found in general search

654 sources found in targeted search

281 sources found in matched FUV targets

In [8]:
def make_variable_target_table(obj_ids):
    variable_table = pd.DataFrame()
    for obj_id in obj_ids:
        eclipse = int(str(obj_id)[-5:])
        #obj_id = int(objid)
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',obj_id)]).to_pandas()
        expt = header_data.loc[header_data['ECLIPSE']==eclipse].loc[header_data['BAND']=='NUV']['EXPT_0']
        cps = this_star["aperture_sum_n_12_8"].tolist()[0]/expt.tolist()[0]
        cps_err = np.sqrt(this_star["aperture_sum_n_12_8"].tolist()[0])/expt.tolist()[0]
        mag = counts2mag(cps,'NUV')
        mag_err_1 = mag-counts2mag(cps+cps_err,'NUV')
        mag_err_2 = counts2mag(cps-cps_err,'NUV')-mag # this one is always larger
        this_star['cps'] = cps
        this_star['cps_err'] = cps_err
        this_star['NUVmag'] = mag
        this_star['NUVmag_err_1'] = mag_err_1
        this_star['NUVmag_err_2'] = mag_err_2
        variable_table = pd.concat([variable_table,
                                 this_star[
                                    ["obj_id","ra","dec","eclipse","NUVmag","NUVmag_err_1","NUVmag_err_2",]]])
    return variable_table


In [46]:
# define manually adjusted source positions
mpt = """06882 99999906882 310.03898  -0.86965 F 
15112 99999915112 173.68275  45.53918 F
29166 99999929166 355.72871   1.65753 F
29390 99999929390  10.92245 -42.34763 U
29643 99999929643  53.06506 -29.51007 U
43201 99999943201 217.15856  57.75332 U
03858 99999903858 135.96049  52.35710 F
04028 99999904028 189.00277  62.49956 F
07940 99999907940   6.43415  17.08139 F
09709 99999909709 164.46273  -3.76613 U
13533 99999913533  16.65372  45.38989 F
13656 99999913656  24.76303 -17.94913 F
16497 99999916497 219.53099  33.10083 F
18081 99999918081 358.46939 -10.54433 F
18380 99999918380   7.70821 -71.84528 F
18753 99999918753  51.26207 -17.21407 F
23505 99999923505  13.11492   0.09855 F
26488 99999926488 212.80185   6.83090 F
28619 99999928619 326.15289   0.70565 F
30122 99999930122  53.00912 -27.04739 F
30668 99999930668 133.73297   2.57409 F
31274 99999931274 120.70829  36.45803 F
36705 99999936705 146.54298  15.54300 F
40521 99999940521  29.19705  -0.35408 F
40930 99999940930  70.63517  18.43965 F
42499 99999942499 165.10157  12.73207 F
43238 99999943238 243.41123  54.66503 F
43828 99999943828 338.34099 -60.93026 F
44684 99999944684 308.13204   7.97132 F
45629 99999945629 160.64052  76.02885 F"""

gfcat_adjusted = pd.DataFrame(
    {'eclipse':np.array(np.array([t.split() for t in mpt.split('\n')])[:,0],dtype='int'),
     'obj_id':np.array(np.array([t.split() for t in mpt.split('\n')])[:,1],dtype='int64'),
     'ra':np.array(np.array([t.split() for t in mpt.split('\n')])[:,2],dtype='float'),
     'dec':np.array(np.array([t.split() for t in mpt.split('\n')])[:,3],dtype='float'),
     'morphology':np.array(np.array([t.split() for t in mpt.split('\n')])[:,4],dtype='str')})

In [47]:
# compile both targeted and untargeted sources
#gfcat_draft = pd.concat([make_variable_target_table(gfcat_corpus['obj_id'].values),
#                         make_variable_target_table(gfcat_targeted['obj_id'].values),
#                         make_variable_target_table(gfcat_fuv_matched['obj_id'].values),
#                         gfcat_adjusted])
#print(len(gfcat_draft))

In [48]:
gfcat_combined = pd.concat([gfcat_corpus,gfcat_targeted,gfcat_adjusted,gfcat_fuv_matched])
print(f"Starting with {len(gfcat_combined)} identified variable source-visits")
gfcat_combined.to_csv('/Users/cm/Downloads/temp.csv')

Starting with 1830 identified variable source-visits

In [49]:
un,ix=np.unique(gfcat_combined['obj_id'],return_index=True)
gfcat_visits = gfcat_combined.iloc[ix].reset_index()

In [76]:
for e in np.unique(gfcat_combined.iloc[ix]['eclipse']):
    visits = gfcat_visits[gfcat_visits['eclipse']==e]
    if len(visits)==2:
        d = angularSeparation(visits['ra'].values.mean(),
                              visits['dec'].values.mean(),
                              visits['ra'].values,visits['dec'].values)
        if all(d<12.5/60/60):
            adj_test = [obj_id.startswith('999999') for obj_id in visits['obj_id'].astype('str').values]
            if any(adj_test):
                # drop the non-adjusted one, which will always be the first one by index because
                # of how we constructed the table => ugly shortcut; don't be like me
                gfcat_visits.drop(visits.index[0],inplace=True)
            # the following judgements were made with manual review
            elif e==9625:
                gfcat_visits.drop(visits.index[0],inplace=True)
            elif e==19732:
                gfcat_visits.drop(visits.index[0],inplace=True)
            elif e==22274:
                gfcat_visits.drop(visits.index[0],inplace=True)
            elif e==27109:
                pass # two flaring stars in the same visit right next to each other!
            elif e==35599:
                gfcat_visits.drop(visits.index[1],inplace=True)
            else:
                print(visits)
                raise
# This results in 5 probably duplicates. I'm going to run these all the way through QA plot gneration
# and then screen them manually. There are probably also other sources with unique obj_id that are basically
# the same source, and those will also be obvious during QA and I can filter them as well.

In [77]:
gfcat_visits

,index,eclipse,obj_id,ra,dec,morphology
0,0,26808,20726808,185.750875,29.544538,T
1,0,21378,39121378,204.877337,30.654018,U
2,0,26045,41026045,149.468637,2.338955,U
3,0,23102,42723102,257.253864,58.873073,F
4,0,17070,44617070,247.108147,40.861124,F
...,...,...,...,...,...,...
1529,5,43201,99999943201,217.158560,57.753320,U
1530,26,43238,99999943238,243.411230,54.665030,F
1531,27,43828,99999943828,338.340990,-60.930260,F
1532,28,44684,99999944684,308.132040,7.971320,F


In [74]:
gfcat_visits[['eclipse','obj_id','ra','dec','morphology']].to_csv(
    'gfcat_visit_table_positions.csv',index=None)